In [1]:
import torch
import numpy as np
device = torch.device("cpu")

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style('white')
#sns.set_style('whitegrid')
import matplotlib
plt.rcParams["axes.grid"] = True
plt.rc('font', family='serif')
#plt.rc('font', weight='bold')

current_palette = sns.color_palette(sns.hls_palette(10+1, l=.4, s=1.0))
sns.palplot(current_palette)
current_palette = np.asarray(current_palette)

In [8]:
import torch
import numpy as np
import math
from itertools import combinations

class spt_manager(object):
    def __init__(self,total_spt,num_Q):
        self.total_spt = total_spt
        self.num_Q = num_Q
        self.covariance_term = False
        

    def k_sm(self,ith_weight,ith_mu,ith_std, tau_collection):            
        exp_term_in = ((tau_collection*ith_std)**2).sum(axis =1,keepdims = True)
        exp_term = np.exp( -2*(math.pi**2)*exp_term_in)
        cos_term_in = (tau_collection*ith_mu).sum(axis =1,keepdims=True)
        cos_term = np.cos(2*math.pi*cos_term_in)
        return ith_weight*(exp_term*cos_term)  

    
    
    def set_tau_collection(self, X, istau = False):
        if torch.is_tensor(X):
            X = X.cpu().data.numpy()

        if istau:
            self.tau_collection = X            
        else :
            self.tau_collection = np.asarray([ith[0]-ith[1] for ith in combinations(X,2)])
        #self.pairwise_tau_collection = combinations(self.tau_collection,2)
                
        ij_taus_sum,ij_taus_minus,i_taus, j_taus = [], [], [], []
        for ith_tau,jth_tau in combinations(self.tau_collection,2):
            ij_taus_sum.append( ith_tau + jth_tau )
            ij_taus_minus.append( ith_tau - jth_tau )
            i_taus.append( ith_tau )
            j_taus.append( jth_tau )
            
        self.ij_taus_sum = np.asarray(ij_taus_sum)
        self.ij_taus_minus = np.asarray(ij_taus_minus)
        self.i_taus = np.asarray(i_taus)
        self.j_taus = np.asarray(j_taus)            
        return 
    


    def g_tau(self, mu,std):
        out = 1 + self.k_sm(ith_weight = 1.0, ith_mu = mu, ith_std = std, tau_collection = 2*self.tau_collection) 
        out += -2*(self.k_sm(ith_weight = 1.0, ith_mu = mu, ith_std = std, tau_collection = self.tau_collection))**2                
        return out
    
    def sum_g_tau(self, mu,std):
        return (self.g_tau(mu,std)).sum(axis = 0)
        
        
    def h_taupair(self, mu,std):    
        out = -self.k_sm(ith_weight = 1.0, ith_mu = mu, ith_std = std, tau_collection = self.i_taus)*self.k_sm(ith_weight = 1.0, ith_mu = mu, ith_std = std, tau_collection = self.j_taus) 
        out += 0.5*self.k_sm(ith_weight = 1.0, ith_mu = mu, ith_std = std, tau_collection = self.ij_taus_sum) 
        out += 0.5*self.k_sm(ith_weight = 1.0, ith_mu = mu, ith_std = std, tau_collection = self.ij_taus_minus)         
        return out
    
    
    def sum_h_taupair(self, mu,std):
        return (self.h_taupair(mu,std)).sum(axis = 0)

    
    
    def calc_sptratio(self, weight_param, mu_param, std_param, iscal_covariance_term):
        nominator_list = []
        for ith_weight,ith_mu,ith_std in zip(weight_param, mu_param, std_param):
            # variance_term
            variance_sum  = .5*self.sum_g_tau(ith_mu,ith_std)
            # covariance_term
            if iscal_covariance_term :
                covariance_sum  = 2*self.sum_h_taupair(ith_mu,ith_std)
            else :
                covariance_sum  = 0
                
            print('variance_sum,covariance_sum')
            print(variance_sum,covariance_sum)
            print('ith_weight*np.sqrt(variance_sum + covariance_sum)')
            print(ith_weight*np.sqrt(variance_sum + covariance_sum))
            # nominator
            nominator_list.append(   ith_weight*np.sqrt(variance_sum + covariance_sum) )
        
        
        ratio = np.asarray(nominator_list)/( np.asarray(nominator_list).sum() + 1e-8)
        
        #print('ratio')
        #print(ratio)
        
        
        
        
        assigned_spt = np.asarray([ int(np.round(ipt)) + 1 for ipt in (self.total_spt-self.num_Q)*ratio]) 
        
        return assigned_spt,ratio
        
            
        

In [ ]:
#X = np.random.randn(10,3)
X = np.arange(0,10,0.05).reshape(-1,1)
X

In [9]:
#X = np.arange(0,100,0.5).reshape(-1,1)


from itertools import combinations    
#tau_collection = np.asarray([ith[0]-ith[1] for ith in combinations(X,2)])
#pairwise_tau_collection = combinations(tau_collection,2)

# ith_weight = [1.2]
# ith_mu = np.array([10.,5.,1.])
# ith_std = np.array([.5,.1,.05])

# ith_weight = [100,0.5,10.,.5,5]
# ith_mu = np.array([[10.,5.,1.],[.4,3.0,1.0],[.4,10.0,1.0]])
# ith_std = np.array([[.5,.1,.05],[.05,.01,.01],[.1,.1,.1]])


ith_weight = [10., 0.5 , 10., .5, 5]
ith_mu = np.array([1,5,10,20,30]).reshape(-1,1)
ith_std = np.array([.0899,.0444,0.0688,.0980,.0177]).reshape(-1,1)


# ith_weight = [1.2]
# ith_mu = np.array([1.2])
# ith_std = np.array([.05])
#out = k_sm(tau_collection,ith_weight,ith_mu,ith_std)

In [10]:
SMspt_manager = spt_manager(total_spt = 500,num_Q = 5)

In [11]:
weightprop = (ith_weight/np.sum(ith_weight))*(SMspt_manager.total_spt)
[int(ith) for ith in weightprop]


[192, 9, 192, 9, 96]

In [12]:
SMspt_manager.set_tau_collection(X,istau=True)

In [13]:
#SMspt_manager.j_taus
assigned_spt_cov,ratio_cov = SMspt_manager.calc_sptratio(weight_param=ith_weight,mu_param=ith_mu,std_param=ith_std , cal_covariance_term = True)
assigned_spt,ratio = SMspt_manager.calc_sptratio(weight_param=ith_weight,mu_param=ith_mu,std_param=ith_std , cal_covariance_term = False)

variance_sum,covariance_sum
[0.8509] [-0.11614271]
ith_weight*np.sqrt(variance_sum + covariance_sum)
[8.57179845]
variance_sum,covariance_sum
[0.39956654] [0.34594276]
ith_weight*np.sqrt(variance_sum + covariance_sum)
[0.4317144]
variance_sum,covariance_sum
[1.12045657] [-0.56251632]
ith_weight*np.sqrt(variance_sum + covariance_sum)
[7.4695398]
variance_sum,covariance_sum
[1.30927171] [-0.46559014]
ith_weight*np.sqrt(variance_sum + covariance_sum)
[0.4592607]
variance_sum,covariance_sum
[0.1326914] [0.11789195]
ith_weight*np.sqrt(variance_sum + covariance_sum)
[2.50291505]
variance_sum,covariance_sum
[0.8509] 0
ith_weight*np.sqrt(variance_sum + covariance_sum)
[9.22442409]
variance_sum,covariance_sum
[0.39956654] 0
ith_weight*np.sqrt(variance_sum + covariance_sum)
[0.31605638]
variance_sum,covariance_sum
[1.12045657] 0
ith_weight*np.sqrt(variance_sum + covariance_sum)
[10.58516212]
variance_sum,covariance_sum
[1.30927171] 0
ith_weight*np.sqrt(variance_sum + covariance_sum)
[0.57211706]

In [14]:
assigned_spt_cov,assigned_spt_cov.sum()

(array([219,  12, 191,  13,  65]), 500)

In [15]:
assigned_spt,assigned_spt.sum()

(array([204,   8, 234,  14,  41]), 501)

In [16]:
ratio

array([[0.40962665],
       [0.01403504],
       [0.4700526 ],
       [0.02540586],
       [0.08087985]])